In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
import seaborn as sns
import ipywidgets as widgets
%matplotlib inline

In [ ]:
data_frame = pd.read_csv("energy_statistics_UE_v3.csv")
data_frame

In [ ]:
#odfiltrowanie wyników dla jednostki kWh w celu sprawdzenia czy da się wyciągnąć procent energii prod z wiatru
#wykres procentowy, nie tylko ilościowy dawałby lepszy kontekst 
df_kWh = data_frame[data_frame["unit"] == "Kilowatt-hours, million"]
df_kWh

In [ ]:
#usuwanie NaN z kolumny transaction
df_kWh = df_kWh[df_kWh["transaction"].notna()] 
df_kWh

In [ ]:
#odfiltrowanie tylko produkcji energii
df_kWh_prod = df_kWh[df_kWh["transaction"].str.contains("production",case = False)]
df_kWh_prod

In [ ]:
#data frame z sumą produkcji energii za dane lata dla całej EU
energy_prod_gr = df_kWh_prod.groupby(by = ["year"], as_index = False)
energy_prod_sum = pd.DataFrame(energy_prod_gr["quantity"].sum())
energy_prod_sum

In [ ]:
#wykres ilościowy produkowanej energii dla całej EU
#widać, że ilość produkowanej energii spada - trzeba zweryfikować czy tak jest i dlaczego :D
#drop po 2008 roku - przez kryzys gospodarczy?

fig, ax = plt.subplots(figsize=(13, 8))
plt.xticks(fontsize=12, rotation = 90)
plt.yticks(fontsize=12)

#ustawianie osi X, żeby wyświetlała wszystkie lata po kolei, a nie co 5.
ax.set_xlim(0, 2014)
ax.xaxis.set_major_locator(MultipleLocator(1))

#tworzenie siatki, żeby wykres był bardziej czytelny
ax.grid(which='major', color='#CCCCCC', linestyle='--')
ax.grid(which='minor', color='#CCCCCC', linestyle=':')

plt.plot(energy_prod_sum[["quantity", "year"]].groupby(by="year").sum(),label="All UE countries")

plt.legend(fontsize=14)
plt.ylabel("Millions of Kilowatts-Hour",fontsize=20)
plt.xlabel('Year',fontsize=20)
plt.title('Total energy production in the EU',fontsize=24)
plt.xlim(1990, 2014)
plt.show()

In [ ]:
#przygotowanie list pod wykresy
list_category = df_kWh["wide_category"].unique()
print(list_category)

list_transaction = df_kWh["transaction"].unique()
print(list_transaction)

In [ ]:
#stworzenie data_framu z całkowitą produkcją roczną energii per kraj - do późniejszych wykresów % udziału
# nuclear + green + gross production z Energy_and_Heat
df_temp = df_kWh_prod.copy()
df_temp

In [ ]:
#przygotowanie wierszy do usunięcia 
to_drop = ['net production', 'total production, autoproducer', 'total production, main activity']
df_temp["to drop"] = np.where(df_temp["transaction"].isin(to_drop), "Y", "N")
df_temp[df_temp["to drop"] == "Y"]

In [ ]:
df_temp = df_temp[df_temp["to drop"] == "N"]
df_temp

In [ ]:
#usunięcie już niepotrzebnej kolumny to drop
df_temp = df_temp.drop(columns = "to drop")
df_temp

In [ ]:
df_temp.to_csv("energy_gross_prod_UE.csv") #zapisanie przygotowanego DF

In [ ]:
#suma energii dla całej EU per rok w rozbiciu na kategorie
cat_prod_sum = df_temp[["wide_category", "year", "quantity"]].groupby(by = ["wide_category", "year"], as_index = False).sum()
cat_prod_sum

In [ ]:
#ogólny wykres dla calej EU dla kategorii z udziałem poszczególnych kategorii
z = cat_prod_sum
z = z[["year", "wide_category", "quantity"]]
z_pivot = z.pivot(index = "year", columns = "wide_category", values = "quantity")
z_pivot = z_pivot.fillna(0)
z_pivot.plot.bar(stacked = True)
plt.title(label = "Share of energy sources in EU")
plt.legend(title=None)
plt.show()

In [ ]:
#ogólny wykres dla calej EU dla kategorii z udziałem poszczególnych kategorii - stack 
z = cat_prod_sum
z = z[["year", "wide_category", "quantity"]]
z_pivot = z.pivot(index = "year", columns = "wide_category", values = "quantity")
z_pivot = z_pivot.fillna(0)
z_pivot.apply(lambda x: x*100/sum(x), axis=1).plot(kind="bar", stacked=True)
plt.title(label = "Share of green energy sources in EU")
plt.legend(title=None)
plt.show()

In [ ]:
#GREEN energy przygotowanie DF
Green_prod = df_kWh_prod[df_kWh_prod["wide_category"] == "Green"]
list_trans_green = Green_prod["transaction"].unique()

In [ ]:
#DF z sumą energii zielonej per rok dla całej UE
green_prod_gr = Green_prod.groupby(by = ["year"], as_index = False)
green_prod_sum = pd.DataFrame(green_prod_gr["quantity"].sum())
green_prod_sum

In [ ]:
#wykres z totalem produkcji energii zielonej w UE w poszczególnych latach
#widać duży skok po 2005 roku można zrobić research czy nie weszła jakaś dyrektywa/dopłaty - ciekawostka do prezentacji
fig, ax = plt.subplots(figsize=(13, 8))
plt.xticks(fontsize=12, rotation = 90)
plt.yticks(fontsize=12)

#ustawianie osi X, żeby wyświetlała wszystkie lata po kolei, a nie co 5.
ax.set_xlim(0, 2014)
ax.xaxis.set_major_locator(MultipleLocator(1))

#tworzenie siatki, żeby wykres był bardziej czytelny
ax.grid(which='major', color='#CCCCCC', linestyle='--')
ax.grid(which='minor', color='#CCCCCC', linestyle=':')

plt.plot(green_prod_sum[["quantity", "year"]].groupby(by="year").sum(),label="All UE countries")

plt.legend(fontsize=14)
plt.ylabel("Millions of Kilowatts-Hour",fontsize=20)
plt.xlabel('Year',fontsize=20)
plt.title('Total green energy production in the EU',fontsize=24)
plt.xlim(1990, 2014)
plt.show()

In [ ]:
#suma energii wyprodukowana w konkretnych latach w podziale na lata i kategorie
gr_green_trans = Green_prod.groupby(by = ["year", "transaction"], as_index = False)
green_trans_sum = pd.DataFrame(gr_green_trans["quantity"].sum())
green_trans_sum

In [ ]:
#wykres z totalem wyprodukowanej energii zielonej w UE w podziale na lata i kategorie  
z = green_trans_sum
z = z[["year", "transaction", "quantity"]]
z_pivot = z.pivot(index = "year", columns = "transaction", values = "quantity")
z_pivot = z_pivot.fillna(0)
z_pivot.plot.bar(stacked = True)
plt.title(label = "Share of green energy sources in EU")
plt.legend(title=None)
plt.show()

In [ ]:
#wykres z totalem wyprodukowanej energii zielonej w UE w podziale na lata i kategorie - stack
z = green_trans_sum
z = z[["year", "transaction", "quantity"]]
z_pivot = z.pivot(index = "year", columns = "transaction", values = "quantity")
z_pivot = z_pivot.fillna(0)
z_pivot.apply(lambda x: x*100/sum(x), axis=1).plot(kind="bar", stacked=True)
plt.title(label = "Share of green energy sources in EU")
plt.legend(title=None)
plt.show()